In [1]:
import sys
sys.path.append("../..")
sys.path.append("../data")
from sources import *
from data.Frame import Frame
from data.Stats import calculate_weighted_average as wa
from geo.countries import Countries
import os
import csv
import json
import urllib.request
from IPython.display import clear_output
c = Countries()
import pandas as pd
from pymongo import MongoClient
client = MongoClient('stats.whereistomas.org', username='root', password='Root!0676')
db = client["stats"]
from sources import *

In [2]:
# Download and extract WEO database from IMF
imf.weo.retrieve()

INFO:root:Retrieving source database: https://www.imf.org/-/media/Files/Publications/WEO/WEO-Database/2021/WEOOct2021all.ashx ...
INFO:root:Source database downloaded to: archive/imf-weo-2021-oct.tsv
INFO:root:Starting extraction of data from: archive/imf-weo-2021-oct.tsv
INFO:root:Number of Indicators included: 45
INFO:root:Number of Countries included: 196
INFO:root:Completed data extraction to data/ directory


In [3]:
ccodes = pd.read_csv("data/country.csv")
ccodes.columns = ['iso3code', 'WEO', 'Name']
data = pd.read_csv("data/values.csv")
data.columns=['iso3code', 'indicator', 'Date', 'Value']
data = pd.merge(data, ccodes, on="iso3code", how="left")
indicators = pd.read_csv("data/indicators.csv")
indicators.columns = ['indicator', 'title', 'description', 'units', 'scale']
data = pd.merge(data, indicators, on="indicator", how="left")
data.columns= ['iso3code', 'indicator', 'Date', 'Value', 'WEO', 'Entity', 'title',
       'description', 'units', 'scale']

In [4]:
indicators_to_retrieve = [{"indicator": "GGXWDG", "aggregation":True, "weight": "None"}]

In [40]:
inds = []
for i in indicators_to_retrieve:
    meta = indicators[indicators["indicator"] == i["indicator"]]
    values = []
    ind = data[data['indicator'] == i['indicator']]
    years = ind['Date'].unique().tolist()
    for ii in range(len(ind)):
        date = str(ind.iat[ii, 2])
        value = str(ind.iat[ii, 3])
        value = value.replace(",", "")
        value = float(value) if any([char.isdigit() for char in value]) else None
        entity = str(ind.iat[ii, 5]).lower()
        values.append({'entity':entity, 'date':date, 'value':value})

    indicator = {
        'name' : i["indicator"],
        'aggregation' : False,
        'weight': "None",
        'description': meta.iat[0, 1],
        'years':[],
        'sourcenote': meta.iat[0, 2],
        'connector': "",
        'source':"IMF",
        'sourceurl': "https://www.imf.org/en/Publications/WEO/weo-database/2021/October",
        'keyField': "entity",
        'pivotField': "date",
        'valueField': "value",
        'data' : values
        }
    inds.append(indicator)

In [41]:
for ind in inds:
    db["indicators"].update_one({ "name" : ind['name']}, 
                                {"$set":{"name" : ind['name'],
                                "aggregation": ind["aggregation"],
                                "weight": ind["weight"],
                                "description" : ind['description'], 
                                "years" : years, 
                                "sourceNote" : ind['sourcenote'],
                                "connector" : ind['connector'],
                                "source" : ind['source'],
                                "sourceurl" : ind['sourceurl'], 
                                "keyField" : "entity",
                                "pivotField" : "date",
                                "valueField" : "value",
                                'data' : ind['data']}}, True)
    print(ind['name'] + " processed.")

GGXWDG processed.


In [ ]:
imf.weo.check_indicators()

In [ ]:
inds[0]['data']

In [ ]:
indicators    